In [1]:
import pymrio
exio3 = pymrio.parse_exiobase3(path="exiobase/IOT_2022_pxp.zip")
import country_converter as coco
cc = coco.CountryConverter()
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Suppress future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
elec_sectors = exio3.Z.filter(like="Electricity", axis=0).index.get_level_values(1).unique().to_list()
renewables_list = ['Electricity by hydro','Electricity by wind','Electricity by biomass and waste','Electricity by solar photovoltaic','Electricity by solar thermal','Electricity by tide, wave, ocean','Electricity by Geothermal']
other_renewables_list = ['Electricity by hydro','Electricity by biomass and waste','Electricity by solar thermal','Electricity by tide, wave, ocean','Electricity by Geothermal']
oilothers_sectors = ["Electricity by petroleum and other oil derivatives","Electricity nec"]
high_emission_sectors = ["Electricity by petroleum and other oil derivatives","Electricity by coal", "Electricity by gas"]
low_emission_sectors = ['Electricity by nuclear','Electricity by hydro','Electricity by wind','Electricity by biomass and waste','Electricity by solar photovoltaic','Electricity by solar thermal','Electricity by tide, wave, ocean','Electricity by Geothermal',"Electricity nec"]

In [6]:
# Setting up the variables
vehicles_list = ["ICEV","PHEV","BEV","FCEV"]
stressor = "Carbon dioxide (CO2) IPCC categories 1 to 4 and 6 to 7 (excl land use, land use change and forestry)"
scenario_list = ["Projection-APS","Projection-STEPS"]
column_list = ["scenario","year","vehicles","stage","region","emission"]
sector_column_list = ["scenario","year","vehicles","stage","region","sector","emission"]

emix_list = ["ori","APS2030"]
demand_list = ['normal','EU100','EU0']
year_list = [2023,2025,2030,2035]


In [7]:
Z_elec = exio3.Z.filter(like="Electricity", axis=0)
A_elec = exio3.A.filter(like="Electricity", axis=0)

In [8]:
A_elec.to_csv("data/outlook_emix_actual.csv") # for comparison purpose in exploration

In [6]:
# read outlook 
# input it as a new A
file_path = "data/outlook_emix_AnnouncedPledgesScenario_2030.csv"
outlook_final = pd.read_csv(file_path,header=[0, 1], index_col=[0, 1])

In [7]:
#update with new A based on the ratio of each electricity product with total total electricity requirement
A_elec_total = A_elec.groupby(level=0).sum()
A_elec_update = outlook_final.mul(A_elec_total)

A = exio3.A.copy()
exio3_ori = exio3.copy()
A.update(A_elec_update) # the new updated A

In [8]:
def calc_emission(vehicles,outlook): 
    exio3.Y.iloc[:,:] = 0
    exio3.Y["EU",vehicles] = outlook[vehicles]
    exio3.Y.fillna(0,inplace=True)
    
    # calculate with new demand
    # coeff : A, L, S, M should be the same with original
    exio3.x = None # reset the output
    exio3.Z = None  # expect new transaction to satisfy the new the output
    exio3.impacts.D_cba = None # reset foot print cba, others are not synch because they consider the emission from final demand
    exio3.impacts.F = None
    exio3.calc_all()
    
    diag_emission = None
    #become a matrix F of regional - sector X regional - sector 
    diag_emission = exio3.impacts.diag_stressor(stressor)
    
    exio3.emission  = diag_emission
    exio3.calc_all()
    
    #print(exio3.meta)
    #print("done calculation for "+vehicles)
    emission = exio3.emission.D_cba
    return emission

In [9]:
def calc_emission_stage(stage):
    # read new outlook 
    file_path = "data/outlook_"+stage+"_"+scenario+".csv"
    if ((stage=="manufacturing") | (stage=="battery")): #manufacturing and battery stage will depend on where the stage will be produced
           file_path = "data/outlook_"+stage+"_"+scenario+"_"+demand+".csv"
    
    #---------------------------------
    outlook = pd.read_csv(file_path)
    outlook.set_index(keys=["year","region","sector"],inplace=True)

    #start collectimg emission data
    emission_stage = pd.DataFrame(columns=column_list)
    emission_sectors = pd.DataFrame(columns=sector_column_list)

    for year in year_list: 
        for vehicles in vehicles_list:
            emission = calc_emission(vehicles,outlook.loc[year])
            
            # group by sectors
            emission_sector = emission.sum(axis=1).to_frame().reset_index()
            emission_sector.rename(columns = {0:"emission"}, inplace=True)
            emission_sector["stage"] = stage
            emission_sector["scenario"] =  scenario
            emission_sector["year"] =  year
            emission_sector["vehicles"] =  vehicles
            ## add to overall record
            emission_sectors = pd.concat([emission_sectors,emission_sector],axis=0)
            
            
            # group by regions
            emission_regions = emission.groupby(level=0).sum()
            emission_regions_total = emission_regions.sum(axis=1).to_frame().reset_index()
            emission_regions_total.rename(columns = {0:"emission"}, inplace=True)
            emission_regions_total["stage"] = stage
            emission_regions_total["scenario"] =  scenario
            emission_regions_total["year"] =  year
            emission_regions_total["vehicles"] =  vehicles
            
            ## add to overall record
            emission_stage = pd.concat([emission_stage,emission_regions_total],axis=0) 
    
    emission_sectors.reset_index(drop=True,inplace=True) 
    emission_sectors.to_csv("outputs/emission_emix_sectors_"+stage+"_"+demand+"_"+emix+".csv",index=False)
   
    #emission_stage.reset_index(drop=True,inplace=True) 
    #emission_stage.to_csv("emission_emix_regions_"+stage+"_"+demand+"_"+emix+".csv",index=False)

In [10]:
exio3.A = A
exio3.Z = None
exio3.calc_system() # calc again whether the Z result consistent with new A

#still using EU vs Non EU
region_agg = coco.agg_conc(original_countries='EXIO3', aggregates=["EU"], missing_countries='Non EU', merge_multiple_string=None)
exio3.aggregate(region_agg = region_agg)
exio3.calc_system()

In [11]:
# calculate every stage using new energy mix 
emix = emix_list[1] # energy mix : APS 2030 
scenario = scenario_list[0] # vehicle sales : APS, 2030
demand = demand_list[0] # vehicle  sales demand : normal based on actual market

calc_emission_stage(stage="manufacturing")
calc_emission_stage(stage="welltotank")
calc_emission_stage(stage="battery")
calc_emission_stage(stage="disposal")

In [12]:
#back to original energy mix
emix = emix_list[0]
exio3 = exio3_ori.copy()
exio3.aggregate(region_agg = region_agg)
exio3.calc_system()

In [13]:
calc_emission_stage(stage="manufacturing")
calc_emission_stage(stage="welltotank")
calc_emission_stage(stage="battery")
calc_emission_stage(stage="disposal")

In [14]:
# combine all the emission emix files
stage_list = ["manufacturing","welltotank","driving","battery","disposal"]
scenario = scenario_list[0] # vehicle sales : APS, 2030
demand = demand_list[0] # vehicle  sales demand : normal based on actual market
year = 2030


# input all emix scenario : APS2030 and original 
emission_stages = pd.DataFrame()
for emix in emix_list:
    print(emix)
    for stage in stage_list:    
        if (stage == "driving") :
            file_path = "outputs/emission_by_stage_"+stage+"_"+demand+".csv" # for driving phase, there is no impact of energy mix, use emission result from previous analysis
        else : 
            file_path = "outputs/emission_emix_sectors_"+stage+"_"+demand+"_"+emix+".csv"
            
        emission_stage = pd.read_csv(file_path)
        
        emission_stage = emission_stage[(emission_stage.year == year) & (emission_stage.scenario == "Projection-APS")]
        emission_stage["emix"] = emix
        emission_stages = pd.concat([emission_stages,emission_stage])
        
emission_stages = emission_stages.groupby(["scenario","year","vehicles","stage","region","emix"]).sum().reset_index()
emission_stages.to_csv( "outputs/emission_emix_all_"+scenario+"_"+demand+"_"+str(year)+".csv",index=False)

ori
APS2030
